## Importing the required libraries

In [24]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding


## Loading the data into a string

In [25]:
with open('poems.txt', 'r') as file:
    text = file.read()


## Mapping 

In [26]:
unique_chars = sorted(set(text))
char_to_index = {char: index for index, char in enumerate(unique_chars)}
index_to_char = {index: char for index, char in enumerate(unique_chars)}


## Pre-processing 

In [27]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length = 36  # You can adjust this based on your preferences
sequences = []
next_chars = []

for i in range(0, len(text) - max_sequence_length, 1):
    input_sequence = text[i:i + max_sequence_length]
    output_char = text[i + max_sequence_length]
    sequences.append([char_to_index[char] for char in input_sequence])
    next_chars.append(char_to_index[output_char])

# Pad sequences to ensure they all have the same length
X = pad_sequences(sequences, maxlen=max_sequence_length, dtype='int32')
y = tf.keras.utils.to_categorical(next_chars, num_classes=len(unique_chars))




## GRU network

In [28]:
model = Sequential([
    Embedding(len(unique_chars), 256, input_length=max_sequence_length),
    GRU(1024, return_sequences=True),
    GRU(1024),
    Dense(len(unique_chars), activation='softmax')
])


model.compile(loss='categorical_crossentropy', optimizer='adam')


## Helper functions

In [29]:
# a) Helper function to sample the next character
def sample_next_char(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# b) Helper function to generate text after each epoch
def generate_text(seed_text, length=400, temperature=0.5):
    generated_text = seed_text
    for _ in range(length):
        encoded_text = [char_to_index[char] for char in generated_text[-max_sequence_length:]]
        encoded_text = np.array([encoded_text])
        predicted_probs = model.predict(encoded_text, verbose=0)[0]
        next_index = sample_next_char(predicted_probs, temperature)
        next_char = index_to_char[next_index]
        generated_text += next_char
    return generated_text

# c) Helper function to save the model after each epoch in which loss decreases
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('model.h5', monitor='loss', verbose=1, save_best_only=True)

# d) Helper function to reduce the learning rate each time the learning plateaus
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.001)


## Training

In [30]:
batch_size = 128
epochs = 15

model.fit(X, y, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint, reduce_lr])


Epoch 1/15
14/14 [==============================] - ETA: 0s - loss: 3.6033
Epoch 1: loss improved from inf to 3.60330, saving model to model.h5
14/14 [==============================] - 48s 3s/step - loss: 3.6033 - lr: 0.0010
Epoch 2/15
14/14 [==============================] - ETA: 0s - loss: 3.1296
Epoch 2: loss improved from 3.60330 to 3.12964, saving model to model.h5
14/14 [==============================] - 46s 3s/step - loss: 3.1296 - lr: 0.0010
Epoch 3/15
14/14 [==============================] - ETA: 0s - loss: 2.8829
Epoch 3: loss improved from 3.12964 to 2.88288, saving model to model.h5
14/14 [==============================] - 45s 3s/step - loss: 2.8829 - lr: 0.0010
Epoch 4/15
14/14 [==============================] - ETA: 0s - loss: 2.6182
Epoch 4: loss improved from 2.88288 to 2.61820, saving model to model.h5
14/14 [==============================] - 45s 3s/step - loss: 2.6182 - lr: 0.0010
Epoch 5/15
14/14 [==============================] - ETA: 0s - loss: 2.4033
Epoch 5: loss

## Generating new and random text

In [31]:
seed_text = "Two roads diverged in a yellow wood,"
generated_poem = generate_text(seed_text, length=400, temperature=0.5)
print(generated_poem)


Two roads diverged in a yellow wood,
And savraveled by,
And that I stood my dreams undergrowth;

Then took that mas mads mingerem bloth-as fare.

The brag with crooked hands;
The blages han
and I ke tast the blothe ave spread my dreams;
I have spread my dreams undergrowth;

The blake all the dif a yellow with a stood,
And sire a tho diverg in the was hade
To way tha firs crag with crooked hands;
Close tast the bloths undergrowth;

T
